In [65]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9VrYo3EhdJ0ckZBNA9xP3AVdAqn0kuHyZpd8pChbn8c&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5w10ycaadGeB8sAKSwPB1VnarSbxNhznS-e6q1sgmKmtiI4jFFj10

Successfully saved authorization token.


In [66]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
from datetime import datetime
import time 


PAIS= ee.FeatureCollection("USDOS/LSIB/2017")\
         .filter(ee.Filter.eq('COUNTRY_NA', 'Mexico'))

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)    
    
def rusle(roi, prefix, folder, scale):
  """ Estima la Erosion mediante RUSLE
  Args:
    roi (ee.Geometry): Ambito de estudio
    prefix: Palabra que se agregara al inicio de cada Imagen a descargar
    folder: Folder en Google Drive donde se exportara los resultados
    scale: Escala de trabajo del proyecto
  """

  #-----------
  # R factor
  #-----------

  clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").clip(PAIS)
  year = clim_rainmap.reduce(ee.Reducer.sum())
  R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
  factorR = R_monthly.reduce(ee.Reducer.sum()).rename('factorR')
  
  #-----------
  # K factor
  #-----------

  # Cargamos toda la informacion necesaria para estimar el factor K
  sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(PAIS)
  silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100).clip(PAIS)
  clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(PAIS)
  morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58).clip(PAIS)
  sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
  orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(PAIS)
  #Juntando todas las imagenes en una sola
  soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )
  factorK = soil.expression(
    '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
    {
      'SAND': soil.select('sand'),
      'SILT': soil.select('silt'),
      'CLAY': soil.select('clay'),
      'MORG': soil.select('morg'),
      'SN1':  soil.select('sn1'),
      'CORG': soil.select('orgcar')
    }).rename('factorK')
  
  #-----------
  # LS factor
  #-----------

  facc = ee.Image("WWF/HydroSHEDS/15ACC").clip(PAIS)
  dem = ee.Image("WWF/HydroSHEDS/03CONDEM").clip(PAIS)
  slope = ee.Terrain.slope(dem)

  ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])

  factorLS = ls_factors.expression(
    '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
    {
      'FACC': ls_factors.select('facc'),
      'SLOPE': ls_factors.select('slope')     
    }).rename('factorLS')
  
  #-----------
  # C factor
  #-----------
  ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").filterBounds(PAIS).median().multiply(0.0001).select('NDVI').clip(PAIS)
  factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431).rename('factorC')

  erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK).rename('A')
  erosion_output= ee.Image([erosion,factorC,factorR,factorLS,factorK])
  
rusle(PAIS, 'RUSLE_', 'RUSLE_MASTERGIS', scale=1000)



centroid = PAIS.geometry().centroid().coordinates().getInfo()

paleta_ero_rusl = ["#00BFBF", "#00FF00", "#FFFF00", "#FF7F00", "#BF7F3F", "#141414"]

RUSLE_pais = {'RUSLE_pais':erosion_output.select('A').getMapId({'min':0,'max': 6000,'paleta':paleta_ero_rusl})}


mapdisplay(centroid,RUSLE,zoom_start= 6)


